source: https://www.kaggle.com/tomras/cnn-classifier-using-keras

In [24]:
import json
import numpy as np
from matplotlib import pyplot as plt
from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from keras.utils.vis_utils import plot_model
import pydot
import load_ship_data as lsd
import time

Note: This notebook assumes keras-gpu is installed instead of the regular keras

Note: You'll need to downgrade to python 3.7 to install keras-gpu

Check for GPUs

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10198986979239463533
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6610781471
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10699102043536263858
physical_device_desc: "device: 0, name: GeForce RTX 2070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


load data

In [51]:
np.random.seed(522)
data_path="data/shipsnet.json"
train,test,valid=lsd.load_data_train_test_split(data_path)
ship_prob=np.sum(train[1])*1.0/train[1].shape[0]
p=np.array([1-ship_prob,ship_prob])
print(p)
Xtrain=train[0]
ytrain=train[1]
Xtest=test[0]
ytest=test[1]
Xvalid=valid[0]
yvalid=valid[1]
#Xtrain=Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3])
#Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1]*Xtest.shape[2]*Xtest.shape[3])
#Xvalid=Xvalid.reshape(Xvalid.shape[0],Xvalid.shape[1]*Xvalid.shape[2]*Xvalid.shape[3])
print(Xtrain.shape)
print(Xtest.shape)
print(Xvalid.shape)
Xtrain=np.concatenate((Xtrain,Xvalid))
Xtrain=Xtrain/255
Xtest=Xtest/255
ytrain=np.concatenate((ytrain,yvalid))
y_train_mono=ytrain
y_test_mono=ytest
ytrain=to_categorical(ytrain,num_classes=2)
ytest=to_categorical(ytest,num_classes=2)
print(Xtrain.shape)
print(ytrain.shape)
print(y_train_mono.shape,Xtrain.shape)

Training Set Data Length:  2800   Label Length:  2800
TestingSet Set Data Length:  600  Label Length:  600
Validation Set Data Length:  600  Label Length:  600
[0.7525 0.2475]
(2800, 3, 80, 80)
(600, 3, 80, 80)
(600, 3, 80, 80)
(3400, 3, 80, 80)
(3400, 2)
(3400,) (3400, 3, 80, 80)


In [52]:
Xtrain=Xtrain.reshape(-1,80,80,3)
Xtest=Xtest.reshape(-1,80,80,3)
print(Xtrain.shape)
print(Xtest.shape)
X_train=Xtrain
X_test=Xtest
y_train=ytrain
y_test=ytest

(3400, 80, 80, 3)
(600, 80, 80, 3)


define CNN

In [58]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=(80, 80, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
    
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), 
              metrics=['accuracy'])

train the model

In [14]:
history = model.fit(X_train, y_train, batch_size=32, epochs=40, validation_split=0.15)
score = model.evaluate(X_test, y_test) 
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Train on 2890 samples, validate on 510 samples
Epoch 1/40
2890/2890 [==============================] - 2s 766us/step - loss: 0.4012 - accuracy: 0.8204 - val_loss: 0.2706 - val_accuracy: 0.9000
Epoch 2/40
2890/2890 [==============================] - 2s 690us/step - loss: 0.1785 - accuracy: 0.9273 - val_loss: 0.1256 - val_accuracy: 0.9392accuracy: 0. - ETA: 0s - loss: 0.2
Epoch 3/40
2890/2890 [==============================] - 2s 690us/step - loss: 0.1018 - accuracy: 0.9668 - val_loss: 0.0511 - val_accuracy: 0.9804
Epoch 4/40
2890/2890 [==============================] - 2s 690us/step - loss: 0.0597 - accuracy: 0.9806 - val_loss: 0.0745 - val_accuracy: 0.9706 - loss: 0.0629 -  - ETA: 0s - loss: 0.0738 - accuracy: 0. - ETA: 0s - loss: 0.0707 - accuracy - ETA: 0s - loss: 0.065
Epoch 5/40
2890/2890 [==============================] - 2s 695us/step - loss: 0.0544 - accuracy: 0.9830 - val_loss: 0.0430 - val_accuracy: 0.9902curacy:  - ETA: 0s - loss: 0.0509 - accuracy: 0. - ETA: 0s - loss: 0.049

In [15]:
print(model.summary())


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 80, 80, 32)        896       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 78, 78, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 39, 39, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 39, 39, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 39, 39, 64)        18496     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 18, 18, 64)       

In [59]:
def accuracy_score(y, y_model):
    # calculate classification overall accuracy and classwise accuracy
    
    assert len(y) == len(y_model)
    classn = len(np.unique(y))       # number of different classes
    correct_all = y == y_model       # all correct classifications
    acc_overall = np.sum(correct_all) / len(y)
    acc_i = np.zeros(classn)
    for i in range(classn):   
        GT_i = y == i                # samples actually belong to class i
        acc_i[i] = (np.sum(GT_i & correct_all) / np.sum(GT_i))
        
    return acc_i, acc_overall
def conf(pred,y):
    T0=sum([1 if x==y and y==0 else 0 for (x,y) in zip(pred,y)])
    T1=sum([1 if x==y and y==1 else 0 for (x,y) in zip(pred,y)])
    F0=sum([1 if x!=y and y==0 else 0 for (x,y) in zip(pred,y)])
    F1=sum([1 if x!=y and y==1 else 0 for (x,y) in zip(pred,y)])
    return np.array([[T0,F0],[F1,T1]])

In [65]:
X=X_train
y=y_train
k=5
kf=StratifiedKFold(n_splits=k,random_state=None)
acc_scores=[]
conf_mat=[]
times=[]
print(X.shape)
print(y.shape)
print(y_train_mono.shape)


(3400, 80, 80, 3)
(3400, 2)
(3400,)
[]


Although normally we could validate a Keras model using cross_val_score from scikit, in order to validate using the same Stratified KFold as our classifiers, we perform the cross validation manually below. cross_val_score can be fed a StratifiedKFold option, but this option is not compatible with the one-hot encoding used by the Kaggle CNN.

In [66]:
for train_index,test_index in kf.split(X,y_train_mono):
    X_tr,X_te=X[train_index,:],X[test_index,:]
    y_tr,y_te=y[train_index],y[test_index]
    y_tr_m,y_te_m=y_train_mono[train_index],y_train_mono[test_index]
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(80, 80, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), 
              metrics=['accuracy'])
    t0=time.time()
    model.fit(X_tr, y_tr, batch_size=32, epochs=40, validation_split=None)
    y_model = model.predict(X_te)
    t1=time.time()
    t=t1-t0
    times.append(t)
    y_model=np.argmax(y_model,axis=1)
    acc_class,acc_overall=accuracy_score(y_te_m, y_model)
    acc_scores.append(acc_overall)
    conf_mat.append(conf(y_model,y_te_m))

Epoch 1/40
2720/2720 [==============================] - 2s 705us/step - loss: 0.4022 - accuracy: 0.82321s - ETA: 0s - loss: 0.4349 - accura
Epoch 2/40
2720/2720 [==============================] - 2s 624us/step - loss: 0.2163 - accuracy: 0.91140s - loss: 0.2162 - accuracy: 
Epoch 3/40
2720/2720 [==============================] - 2s 628us/step - loss: 0.1123 - accuracy: 0.95741s - ETA: 0s - loss: 0.1141 - accuracy: 
Epoch 4/40
2720/2720 [==============================] - 2s 626us/step - loss: 0.0707 - accuracy: 0.97791s - loss: 0.0532 - ac - ETA: 0s -
Epoch 5/40
2720/2720 [==============================] - 2s 633us/step - loss: 0.0451 - accuracy: 0.98460s - loss: 0.0538 - accu - ETA: 0s - loss: 0.045
Epoch 6/40
2720/2720 [==============================] - 2s 630us/step - loss: 0.0362 - accuracy: 0.9890
Epoch 7/40
2720/2720 [==============================] - 2s 627us/step - loss: 0.0289 - accuracy: 0.98971s - loss: 0.0375 - accuracy: 0.98 - ETA: 1s - loss: 0.0358 - ac - ETA: 0s - loss: 0.

2720/2720 [==============================] - 2s 645us/step - loss: 7.0935e-04 - accuracy: 1.00000s - loss: 9.019
Epoch 22/40
2720/2720 [==============================] - 2s 650us/step - loss: 0.0013 - accuracy: 0.9996
Epoch 23/40
2720/2720 [==============================] - 2s 648us/step - loss: 0.0013 - accuracy: 1.0000 - ETA: 0s - loss: 0.0012 - accuracy: 1.00
Epoch 24/40
2720/2720 [==============================] - 2s 646us/step - loss: 4.2774e-04 - accuracy: 1.00000s - loss: 4.7708e-04 - accu
Epoch 25/40
2720/2720 [==============================] - 2s 654us/step - loss: 4.5824e-04 - accuracy: 1.00000s - loss: 5.0222e-04 - ac
Epoch 26/40
2720/2720 [==============================] - 2s 653us/step - loss: 4.9424e-04 - accuracy: 1.00000s - loss: 7.0591e-04 - ac - ETA: 0s - loss: 5.8713e-04 - accura
Epoch 27/40
2720/2720 [==============================] - 2s 653us/step - loss: 2.9334e-04 - accuracy: 1.00000s - loss: 3.1118e-04 - accuracy: 1.
Epoch 28/40
2720/2720 [======================

2720/2720 [==============================] - 2s 655us/step - loss: 0.0012 - accuracy: 0.9993: 1s - loss: 0.0022 - accuracy: 0.99 - ETA: 1s - loss: 0.0020 - accuracy: 0.99 - ETA: 0s - loss: 0.0019 - accura - ETA: 0s - loss: 0.0016 - accuracy: 0.99 - ETA: 0s - loss: 0.0015 -  - ETA: 0s - loss: 0.0012 - accuracy: 0.99
Epoch 29/40
2720/2720 [==============================] - 2s 647us/step - loss: 2.4646e-04 - accuracy: 1.00000s - loss: 2.9318e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.7458e-04 - accu - ETA: 0s - loss: 2.4602e-04 - ac - ETA: 0s - loss: 2.5392e-04 - accuracy: 1.00
Epoch 30/40
2720/2720 [==============================] - 2s 647us/step - loss: 2.8961e-04 - accuracy: 1.00000s - loss: 3.5383e-04 - accuracy:  - ETA: 0s - loss: 3.6306e
Epoch 31/40
2720/2720 [==============================] - 2s 649us/step - loss: 1.5022e-04 - accuracy: 1.00001s - loss: 2.0973e-04 - accuracy: 1. - ETA: 1s - loss: 1.5355e-04 - ac - ETA: 0s - loss: 1.4983e-04 - accuracy:  - ETA: 0s - loss: 1.885
Epoch 

2720/2720 [==============================] - 2s 659us/step - loss: 6.3779e-04 - accuracy: 1.0000
Epoch 34/40
2720/2720 [==============================] - 2s 663us/step - loss: 6.1935e-04 - accuracy: 1.0000 loss: 0.0011 - accura - ETA: 1s - loss: 6.8785e-04 - accu - ETA: 0s - loss: 6.1653e-04 - accuracy - ETA: 0s - loss: 5.7706e-04 - ac
Epoch 35/40
2720/2720 [==============================] - 2s 665us/step - loss: 0.0018 - accuracy: 0.9993TA: 1s - - ETA: 0s - loss: 0.0018 - accuracy: 0.99 - ETA: 0s - loss: 0.0019 - accuracy: 
Epoch 36/40
2720/2720 [==============================] - 2s 670us/step - loss: 9.9232e-04 - accuracy: 0.9996
Epoch 37/40
2720/2720 [==============================] - 2s 661us/step - loss: 4.1993e-04 - accuracy: 1.0000
Epoch 38/40
2720/2720 [==============================] - 2s 659us/step - loss: 2.8361e-04 - accuracy: 1.00001s - loss: 3.0737e-04 - accuracy: 1.00 - ETA: 1s - l
Epoch 39/40
2720/2720 [==============================] - 2s 665us/step - loss: 2.8103e-04 

2720/2720 [==============================] - 2s 661us/step - loss: 1.0168e-04 - accuracy: 1.00001s - loss: 4.1566e-0 - ETA: 0s - loss: 4.9847e-05 - accura
Epoch 39/40
2720/2720 [==============================] - 2s 652us/step - loss: 9.8299e-05 - accuracy: 1.00001s - loss: 8.3670e-05 -  - ETA: 0s - loss: 8.4028e-0
Epoch 40/40
2720/2720 [==============================] - 2s 662us/step - loss: 2.6808e-04 - accuracy: 1.0000


In [67]:
avg_acc=np.sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
print('Average Runtime = {}'.format(np.sum(np.array(times))*1.0/5))
class0_acc=np.array([x[0][0]*1.0/(x[0][0]+x[0][1]) for x in conf_mat])
class0_acc=np.sum(class0_acc)*1.0/5
class1_acc=np.array([x[1][1]*1.0/(x[1][0]+x[1][1]) for x in conf_mat])
class1_acc=np.sum(class1_acc)*1.0/5
print(class0_acc)
print(class1_acc)

Accuracy from each fold =  [0.9926470588235294, 0.9897058823529412, 0.9941176470588236, 0.9882352941176471, 0.9941176470588236]
Average Accuracy = 0.991764705882353
Average Runtime = 71.82766485214233
0.9937254901960785
0.9858823529411765
